# Projet d'initiation aux techniques de l'Intelligence Artificielle

## Objectifs pédagogiques

Pour achever le module, il faut avoir gagné en autonomie dans la mise en œuvre des techniques de l'IA.

Pour cette raison, les choix sont libres et les solutions proposés ne sont qu'indicatives.

Ce projet a pour but de permettre de comprendre que l'IA ne se réduit pas à des bouts de code utilisant *scikit-learn* ou *keras* dans un Notebook *Jupyter* : ce n'est pas le doigt qu'il faut regarder lorsqu'on montre la lune !

## Spécification

On veut développer une application qui permet d'identifier si une image contient :
- un tournesol
- un chou-fleur
- autre chose

Les images doivent pouvoir faire 128×128 pixels, mais aussi être plus grandes ou plus petites.

## jeu de données

Il est possible d'utiliser n'importre quel jeu de données, notamment :
- [Chou-fleurs](https://data.mendeley.com/datasets/t5sssfgn2v/3)
- [Tournesols](https://cours.iut-orsay.fr/mod/resource/view.php?id=54534&forceview=1)

## Modalités

Le projet doit être réalisé en trinômes. Il faudrait qu'au moins un membre du trinôme dispose d'un ordinateur pouvant servir à entrainer un réseau de neurones pendant plusieurs heures (la nuit ?).

## Attendus

Les trinômes doivent produire :

- un compte-rendu de développement
- un Notebook en Python permettant de produire un modèle de reconnaissance d'image entrainé selon les spécification sous forme d'un ou plusieurs fichier(s).
- une application qui utilise le modèle entrainé pour traiter une image (nom de fichier en argument pour les programmes C++ ou Java, fichier chargé dans la page Web pour une application javascript) et afficher le résultat de l'identification avec un coefficient de certitude sur la classe identifiée.

## Technologies de déploiement

Dans un projet utilisant de l'IA, l'objectif est d'avoir une application déployée, qui ne sera généralement sous forme de Notebook.

On peut classer les technologies d'implémentation d'applications informatiques selon leur degré d'abstraction par rapport à la plateforme d'exécution (matériel informatique et système d'exploitation), un plus haut niveau d'abstraction permettant généralement un déploiement plus facile mais au prix de la performance en exécution.

On choisira alors les technologies en fonction du compromis correspondant aux contraintes de l'application.
Par exemple, on aura le choix entre C++, Java et Javascript, avec C++ > Java > Javascript en performance et Javascript > Java > C++ en facilité de éploiement :
- Javascript : il suffit d'aller sur une page Web pour que le navigateur récupère l'applicationn web
- Java : il faut généralement installer un Jar et avoir le JRE installé
- C++ : il faut installer l'exécutable correspondant à la plateforme (CPU, architecture, système d'exploitation)

Pour ce projet, vous êtes libre de choisir la technologie d'implémentation de l'application parmi Javascript (dans une page web), Java (fichier JAR) ou C++ (fournir un exécutable linké statiquement pour Linux x86_64).

Vous êtes aussi libre de choisir la bibliothèque d'implémentation de réseau de neurones de l'application. [OpenCV](https://opencv.org/platforms/) dispose d'un module [dnn](https://docs.opencv.org/4.x/d2/d58/tutorial_table_of_content_dnn.html) qui permet d'implémenter un réseau de neurones profond en C++ ou en Java. [OpenCV.js](https://docs.opencv.org/4.8.0/df/d0a/tutorial_js_intro.html) permet [la même chose dans le navigateur](https://docs.opencv.org/4.x/d5/d86/tutorial_dnn_javascript.html).

## Technologies d'apprentissage

Il est fortement recommandé d'utiliser un réseau de neurones déjà entrainé et de faire du *transfert learning* et du *fine tuning*. Pour choisir le réseau de neurones, on peut prendre en compte des contraintes de déploiement (cf. taille du réseau de neurones) et les types de couches de neurones utilisées.
En effet, toutes les sortes de neurones ne sont pas forcément implémentées par la bibliothèque choisie pour le déploiement, même s'[il est toujours possible d'implémenter soi-même de nouvelles sortes de neurones](https://docs.opencv.org/4.x/dc/db1/tutorial_dnn_custom_layers.html).

Avant de se lancer dans l'apprentissage utilisant un réseau de neurones pré-entrainé, il est prudent de vérifier que l'on pourra bien charger celui-ci dans l'application.

Il est aussi fortement conseillé d'utiliser l'augmentation de doonnées.

Vous pouvez utiliser *Keras*, mais ce n'est pas obligatoire.




## Transfer learning
Il faut ensuite définir et instancier un réseau de neurones pour faire du *transfer learning*.

Comme on l'a vu, il est souvent préférable d'ajouter de la *régularisation* au modèle qui ava être utilisé pour faire du *transfer learning* :

In [1]:
from keras.regularizers import l2

def add_regularization(model, regularizer=l2(0.0001)):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:

        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

## Choix du réseau pré-entrainé

Il faut en fait choisir celui-ci en fonction des capacités de la bibliothèque qui sera utilisée pour le déploiement afin d'être sûr qu'on pourra charger et utiliser le modèle.
(Utiliser les versions les plus récentes et faires des essais avant de passer du temps à entrainer !)

In [67]:
import os
import tempfile
import keras
from keras.models import model_from_json
from keras.regularizers import l2
import tensorflow as tf

from keras import applications
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, Flatten

def build_mobilenet_transfert(width, height, depth, num_classes):
    (input_shape, chan_dim)=  ((depth, height, width), 1) if keras.backend.image_data_format() == "channels_first" \
    else ((height, width, depth), -1)
    new_input = Input(shape=input_shape)
    base_model = applications.MobileNetV3Small(input_shape=(224, 224, 3),
                                                     alpha=1.0,
                                                    minimalistic= True,
                                                    include_top=False,
                                                    weights="imagenet",
                                                    dropout_rate=0.2,
                                                    include_preprocessing=True)
    base_model = add_regularization(base_model)
    add_model = Sequential()
    add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    add_model.add(Dense(256, activation='relu'))
    add_model.add(Dense(num_classes, activation='softmax'))
    model = keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
    for layer in model.layers[:-5]:
        layer.trainable = False
    return model

model_mobilenet= build_mobilenet_transfert(224,224,3,3)


## Jeu de données

[constituer un jeu de données à partir des fichiers dans les répertoires](https://keras.io/api/data_loading/image/#imagedatasetfromdirectory-function)


In [57]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.applications.mobilenet_v3 import preprocess_input

def load_images_from_directory(directory):
    images = []
    labels = []
    class_names = os.listdir(directory)

    for class_name in class_names:
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                img_path = os.path.join(class_path, filename)
                
                # Vérifier si l'image peut être lue correctement
                img = cv2.imread(img_path)
                if img is not None and not img.size == 0:
                    img = cv2.resize(img, (224, 224))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    images.append(img)
                    labels.append(class_name)
                else:
                    print(f"Impossible de lire l'image : {img_path}")

    return np.array(images), np.array(labels)

# Chargement des images depuis le répertoire src/tournesol
images, labels = load_images_from_directory('ProjetIA/src/data')

# Prétraitement des images
images_preprocessed = preprocess_input(images)

# Conversion des étiquettes en one-hot encoding
label_binarizer = LabelBinarizer()
labels_one_hot = label_binarizer.fit_transform(labels)

# Division des données
X_train, X_temp, y_train, y_temp = train_test_split(images_preprocessed, labels_one_hot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Affichage des dimensions des ensembles
print("Dimensions de l'entraînement:", X_train.shape, y_train.shape)
print("Dimensions de validation:", X_val.shape, y_val.shape)
print("Dimensions de test:", X_test.shape, y_test.shape)


Impossible de lire l'image : ProjetIA/src/data/autre_chose/.DS_Store
Impossible de lire l'image : ProjetIA/src/data/autre_chose/flowers
Dimensions de l'entraînement: (834, 224, 224, 3) (834, 3)
Dimensions de validation: (179, 224, 224, 3) (179, 3)
Dimensions de test: (179, 224, 224, 3) (179, 3)


In [58]:
labels_one_hot.shape

(1192, 3)

In [59]:
labels

array(['chou_fleur', 'chou_fleur', 'chou_fleur', ..., 'tournesol',
       'tournesol', 'tournesol'], dtype='<U11')

## Entrainement
Entrainer le modèle.

In [60]:
y_train

array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [69]:
from keras.optimizers import Adam

# Compiler le modèle
model_mobilenet.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model_mobilenet.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)

# Évaluer le modèle sur l'ensemble de test
test_loss, test_accuracy = model_mobilenet.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/10
27/27 [==============================] - 4s 102ms/step - loss: 0.6568 - accuracy: 1.0000 - val_loss: 0.8630 - val_accuracy: 0.9944
Epoch 2/10
27/27 [==============================] - 2s 86ms/step - loss: 0.6655 - accuracy: 0.9976 - val_loss: 0.8257 - val_accuracy: 0.9944
Epoch 3/10
27/27 [==============================] - 2s 84ms/step - loss: 0.6639 - accuracy: 0.9988 - val_loss: 0.7625 - val_accuracy: 0.9944
Epoch 4/10
27/27 [==============================] - 2s 85ms/step - loss: 0.6476 - accuracy: 0.9976 - val_loss: 0.8973 - val_accuracy: 0.9888
Epoch 5/10
27/27 [==============================] - 2s 86ms/step - loss: 0.9221 - accuracy: 0.9880 - val_loss: 0.7738 - val_accuracy: 0.9888
Epoch 6/10
27/27 [==============================] - 2s 84ms/step - loss: 0.7219 - accuracy: 0.9940 - val_loss: 0.7860 - val_accuracy: 0.9944
Epoch 7/10
27/27 [==============================] - 2s 85ms/step - loss: 0.6519 - accuracy: 0.9988 - val_loss: 0.7814 - val_accuracy: 0.9944
Epoch 8/10
2

## Sauvegarde du modèle entrainté

In [70]:
model_mobilenet.save("mobilenet_with-preprocessing.h5")

/Users/elhadjsylla/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Conversion de la sauvegarde en un format pour l'inférence

Pour utilise le modèle entrainé en inférence seule, en phase de déploiement, on fait une conversion du fichier :

In [71]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.keras.models import load_model
import numpy as np                    
#path of the directory where you want to save your model
frozen_out_path = './freez/'             # name of the .pb file
frozen_graph_filename = "mobilenet_with-preprocessing" #“frozen_graph”

model = load_model("./mobilenet_with-preprocessing.h5")#load_model("./cls_vgg16_6_cl.h5")
# model = ""# Your model# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
    print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)# Save frozen graph to disk
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)# Save its text representation
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                logdir=frozen_out_path,
                name=f"{frozen_graph_filename}.pbtxt",
                as_text=True)


2023-12-15 16:00:55.212396: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-12-15 16:00:55.213467: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


------------------------------------------------------------
Frozen model layers: 
x
------------------------------------------------------------
model_13/rescaling_14/Cast/x
------------------------------------------------------------
model_13/rescaling_14/Cast_1/x
------------------------------------------------------------
model_13/Conv/Conv2D/ReadVariableOp/resource
------------------------------------------------------------
model_13/Conv/BatchNorm/ReadVariableOp/resource
------------------------------------------------------------
model_13/Conv/BatchNorm/ReadVariableOp_1/resource
------------------------------------------------------------
model_13/Conv/BatchNorm/FusedBatchNormV3/ReadVariableOp/resource
------------------------------------------------------------
model_13/Conv/BatchNorm/FusedBatchNormV3/ReadVariableOp_1/resource
------------------------------------------------------------
model_13/expanded_conv/depthwise/pad/Pad/paddings
------------------------------------------

'./freez/mobilenet_with-preprocessing.pbtxt'

In [3]:
import os
import cv2
import numpy as np
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from keras.preprocessing import image
import tensorflow as tf


# Charger le modèle pré-entraîné MobileNetV2
model_mobilenet = tf.keras.models.load_model('freez/mobilenet_with-preprocessing.pb')

# Charger une image du dossier "chou_fleur"
image_path = 'ProjetIA/src/data/chou_fleur/No disease. (185).jpg'  # Remplacez 'your_image.jpg' par le nom de votre image
img = cv2.imread(image_path)
img = cv2.resize(img, (224, 224))  # Assurez-vous que la taille correspond à celle utilisée pendant l'entraînement
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_array = np.expand_dims(img, axis=0)  # Ajouter une dimension pour créer un lot (batch)

# Prétraitement de l'image
img_array = preprocess_input(img_array)

# Faire la prédiction
predictions = model_mobilenet.predict(img_array)

# Décoder et afficher les prédictions
decoded_predictions = decode_predictions(predictions)
for i, (imagenet_id, label, score) in enumerate(decoded_predictions[0]):
    print(f"{i + 1}: {label} ({score:.2f})")

# Afficher l'image
cv2.imshow('Image', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


OSError: Unable to open file (file signature not found)

## Déploiement


Pour déployer le réseau de neurone dans une application cliente, il faut choisir les technologies de déploiement, notamment le langage (ici, Javascript, Java ou C++) et la bibliothèque (par exemple OpenCV ou TensorFlow).

## Exemple en C++

In [235]:
cxx_prog=r"""
//#I=/usr/include/opencv4/
#include <fstream>
#include <sstream>
#include <iostream>
#include <opencv2/dnn.hpp>
#include <opencv2/imgproc.hpp>
#include <opencv2/videoio.hpp>

using namespace cv;
using namespace dnn;
std::vector<std::string> classes;
int main(int argc, char** argv)
{
    float scale(1.f);
    Scalar mean(123, 117, 104);
    bool swapRB = true;
    int inpWidth = 224;
    int inpHeight = 224;
    String model = "freez/mobilenet_with-preprocessing.pb";//"./tensorflow_inception_graph.pb";
    String config = "";
    String framework = "";
    int backendId(0);
    int targetId (0);
    // Open file with classes names.
    std::string file = "./mobilenet-imagenet_labels.txt";
    std::ifstream ifs(file.c_str());
    if (!ifs.is_open())
        {CV_Error(Error::StsError, "File " + file + " not found");}
    std::string line;
    while (std::getline(ifs, line))
    {
        classes.push_back(line);
    }
    CV_Assert(!model.empty());
    Net net = readNet(model, config, framework);
    net.setPreferableBackend(backendId);
    net.setPreferableTarget(targetId);
    VideoCapture cap;
    cap.open("generated__0_9963.jpg");
    // Process frames.
    Mat frame, blob;
    cap >> frame;
    blobFromImage(frame, blob, scale, Size(inpWidth, inpHeight), mean, swapRB, false);
    net.setInput(blob);
    Mat prob = net.forward();
    Point classIdPoint;
    double confidence;
    minMaxLoc(prob.reshape(1, 1), 0, &confidence, 0, &classIdPoint);
    int classId = classIdPoint.x;
    // Put efficiency information.
    std::vector<double> layersTimes;
    double freq = getTickFrequency() / 1000;
    double t = net.getPerfProfile(layersTimes) / freq;
    std::string label = format("Inference time: %.2f ms", t);
    std::cout<<label<<std::endl;
    label = format("%s: %.4f", (classes.empty() ? format("Class #%d", classId).c_str() 
                                                : classes[classId].c_str()),
                                confidence);
        std::cout<< label<<std::endl;
    return 0;
}

"""
with open("exemple.cxx", "w") as text_file:
    text_file.write(cxx_prog)


(Les *flags* indiquant les chemins où OpenCV est installé doivent être adaptés)

In [236]:
!g++ exemple.cxx -I/usr/local/include/opencv4/ -L/usr/local/lib/ -o exemple_cxx -lopencv_core -lopencv_videoio -lopencv_imgcodecs -lopencv_dnn

In [237]:
! export LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH;./exemple_cxx

Inference time: 32.69 ms
tench, Tinca tinca: 0.2725


## Java
On peut [aussi faire la même chose en Java](https://learnopencv.com/image-classification-with-opencv-java/)

## Javascript

On peut aussi charger le modèle et l'utiliser en inférence en Javascript à partir d'une page web :

In [126]:
from IPython.display import IFrame
IFrame('https://docs.opencv.org/3.4/js_image_classification.html', width=700, height=350)
